# Getting Zooniverse Participates

Looked to see how many people rated an image as malicious and non-malicious.

In [1]:
#Libraries 
import os
#import openai
import psycopg2 #for database connection
import pandas as pd 
import sqlalchemy
import numpy as np
import matplotlib.pyplot as plt # for plotting
import json
import re

# Kruskal-Wallis analysis of variance
import scipy.stats as ss # For Kruskal-Wallis test
import scikit_posthocs as sp #For post hoc tests. 
from termcolor import colored # for coloring the print text
import warnings # to ignore plot warnings

# For ggplot
from plotnine import *

# For bolding the printed text
from termcolor import colored

In [3]:
# Define database
hostname = 'localhost'
# f = open("Username.txt", "r")
# username = f.read()
# password = f.read()
database = 'phishdatabase'
username = 'postgres'
password = 'postgres'
port = "5432"

In [4]:
# Create Connection
try:
    connection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database, port=port )
    
    
except:
    print("I am unable to connect to the database")

In [5]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 10)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 32)

In [6]:
# Step 1
# Getting gold standard images from phase 1
gold_std = pd.read_sql_query("select subject_id_ph1 as subject_id, subject_id_ph1, filename, malicious, gold_std from cybertrust_zooniverse_datamatch where gold_std is true", connection)
gold_std

gold_std = gold_std.dropna(subset= ['subject_id']).reset_index()
gold_std = gold_std.drop('index', axis=1)
gold_std

C:\Users\#3\AppData\Local\Temp\ipykernel_20276\3301370320.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


,subject_id,subject_id_ph1,filename,malicious,gold_std
0,49251675.0,49251675.0,UPRailRoad.png,False,True
1,49251676.0,49251676.0,TaxInformationEmail.png,True,True
2,49251677.0,49251677.0,Paypal.png,True,True
3,49251678.0,49251678.0,Newsweek.png,False,True
4,49251679.0,49251679.0,whatsup.png,True,True
5,49251680.0,49251680.0,VirusSpyware.png,True,True
6,49251681.0,49251681.0,VenmoUber.png,False,True
7,49251682.0,49251682.0,venmo.png,False,True
8,49251683.0,49251683.0,Vanguard.png,False,True
9,49251684.0,49251684.0,USPSGmail.png,True,True


In [13]:
sql = """select c.subject_ids as subject_id, b.annotations->> 'value' as phishing_class 
       from zooniverse_phish_classifications as c, 
    jsonb_array_elements(c.annotations) as b(annotations)
    where  b.annotations->>'task' = 'T3'
"""

zooniverseclassification = pd.read_sql_query(sql, connection)
zooniverseclassification

#zooniverseclassification = zooniverseclassification[zooniverseclassification['user_answers'] == '[]']
#zooniverseclassification[zooniverseclassification['subject_id'] == 43857067]
zooniverseclassification

C:\Users\#3\AppData\Local\Temp\ipykernel_20276\1184310123.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


,subject_id,phishing_class
0,43857001,Nothing Phishy Here
1,43857012,Nothing Phishy Here
2,43857072,Nothing Phishy Here
3,43857007,Nothing Phishy Here
4,43857095,None
...,...,...
31364,52844553,None
31365,52846983,None
31366,52846998,None
31367,52846937,None


In [14]:
zooniverseclassification["phishing_class"].unique()

array(['Nothing Phishy Here', None, "Something's Phishy"], dtype=object)

In [16]:
Zoon_malicious= zooniverseclassification.groupby('subject_id').apply(lambda x: pd.Series(
    dict( Total_People_who_put_malicious = (x.phishing_class == "Something's Phishy").sum(),
          No_malicious = (x.phishing_class == "Nothing Phishy Here").sum()
    ))).reset_index('subject_id')

change_type ={"subject_id": "int64"}
Zoon_malicious = Zoon_malicious.astype(change_type)
data_types = Zoon_malicious.dtypes
data_types

match = pd.merge(gold_std, Zoon_malicious, on="subject_id")
match   

C:\Users\#3\AppData\Local\Temp\ipykernel_20276\3885703675.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


,subject_id,subject_id_ph1,filename,malicious,gold_std,Total_People_who_put_malicious,No_malicious
0,49251675.0,49251675.0,UPRailRoad.png,False,True,10,6
1,49251676.0,49251676.0,TaxInformationEmail.png,True,True,11,5
2,49251677.0,49251677.0,Paypal.png,True,True,12,5
3,49251678.0,49251678.0,Newsweek.png,False,True,1,14
4,49251679.0,49251679.0,whatsup.png,True,True,12,4
5,49251680.0,49251680.0,VirusSpyware.png,True,True,7,7
6,49251681.0,49251681.0,VenmoUber.png,False,True,3,12
7,49251682.0,49251682.0,venmo.png,False,True,2,14
8,49251683.0,49251683.0,Vanguard.png,False,True,11,3
9,49251684.0,49251684.0,USPSGmail.png,True,True,15,0


In [17]:
#match.to_csv("People.csv")